# 5年营业收入增长率 sales_growth
过去5个财年的 每股营业收入增长 除以 年均每股营业收入

注意：对于基本面数据，
每年11-第二年4月使用前一年0930的TTM数据，既三季报+前一年四季度
5-8月使用1季度0331的TTM数据，
9-10月使用中报0630的TTM数据，
11-12月使用当年三季报0930的TTM数据

In [2]:
import datetime

import pandas as pd
# 获取tushare每股营业收入数据
import tushare as ts

pro = ts.pro_api('e9628663b45f87ea92e39aaa7127063830ebd5090bc1e8943138b84f')



In [3]:
datetime.datetime.today().strftime('%Y%m%d')

'20230405'

In [4]:
df = pro.fina_indicator_vip(period='20221231')
df

,ts_code,ann_date,end_date,eps,dt_eps,total_revenue_ps,revenue_ps,capital_rese_ps,surplus_rese_ps,undist_profit_ps,...,roe_yoy,bps_yoy,assets_yoy,eqt_yoy,tr_yoy,or_yoy,q_sales_yoy,q_op_qoq,equity_yoy,update_flag
0,A22231.SZ,20230118,20221231,4.1400,4.0500,27.3891,27.3891,1.4895,0.5000,6.4809,...,62.8621,74.3120,62.2231,74.3135,58.6078,58.6078,NaN,NaN,74.3135,1
1,A22460.SZ,20230118,20221231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.1659,NaN,45.6121,25.5637,91.0482,91.0482,NaN,NaN,25.5637,1
2,A22278.SZ,20230130,20221231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-11.0430,NaN,30.7480,41.9701,21.4938,21.4938,NaN,NaN,41.9701,0
3,A22137.BJ,20230201,20221231,0.4500,0.4500,13.0174,13.0174,1.6935,0.1906,0.8117,...,4.2923,4.7087,-1.5606,4.7099,15.8528,15.8528,40.7736,477.0235,4.7099,1
4,A21410.SZ,20230310,20221231,0.2700,0.2700,2.6092,2.6092,0.2330,0.0900,0.4380,...,43.0436,17.3300,4.2451,17.7526,36.9964,36.9964,3.2901,41.0361,17.7526,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,300956.SZ,20230405,20221231,-0.2100,-0.1500,10.3967,10.3967,4.4384,0.0981,2.3899,...,-141.3840,-3.7802,-4.7616,6.2800,-18.7776,-18.7776,-50.7755,-213.0578,6.2800,1
1804,A23053.SH,20230404,20221231,1.3826,1.3826,29.1154,29.1154,7.8299,0.0560,1.4079,...,124.6079,15.6360,34.3165,15.6355,75.4870,75.4870,NaN,NaN,15.6355,1
1805,A23054.SZ,20230404,20221231,1.0583,1.0583,3.7025,3.7025,2.2736,0.2097,2.1022,...,48.0619,25.2870,22.7006,25.2858,43.2791,43.2791,NaN,NaN,25.2858,1
1806,A22319.BJ,20230405,20221231,0.9400,0.9400,2.2881,2.2881,1.4156,0.4986,1.4546,...,-5.2598,10.3500,5.0483,10.3123,-0.7944,-0.7944,-6.2946,-20.1931,10.3123,0


In [13]:
# 获取tusahre财务指标数据，可指定数据品种、证券范围以及数据获取时间
def get_fin_indicators(period, indicators, security=[], date=datetime.datetime.today().strftime('%Y%m%d'),
                       market_sign='A'):
    '''
    :param security: security list
    :param date: yyyymmdd
    :param period: yyyymmdd
    :param indicators: list
    :return: dataframe
    '''

    fields = ['ts_code', 'ann_date', 'end_date', 'update_flag']
    fields.extend(indicators)
    revenue_ps_df = pro.fina_indicator_vip(period=period, fields=fields)
    revenue_ps_df.dropna(inplace=True)
    # 查看是否指定了证券区间
    if len(security) > 0:
        revenue_ps_df = revenue_ps_df.loc[revenue_ps_df['ts_code'].isin(security)]
    if market_sign == 'A':
        revenue_ps_df = revenue_ps_df[revenue_ps_df['ts_code'].str.startswith(('0', '3', '6'))]
    revenue_ps_df = revenue_ps_df[revenue_ps_df['ann_date'] <= date]
    revenue_ps_df = revenue_ps_df.reset_index()
    revenue_ps_df.drop('index', axis=1)
    return revenue_ps_df

# get_fin_indicators(date='20230316', period='20221231', indicators=['eps', 'revenue_ps'])

In [14]:
# 生成revenue-ps的TTM数据
period = '20221231'
indicators = ['eps', 'revenue_ps']
get_fin_indicators(period, indicators)

,index,ts_code,ann_date,end_date,eps,revenue_ps,update_flag
0,10,001311.SZ,20230206,20221231,4.2000,31.6510,0
1,11,001225.SZ,20230206,20221231,4.5500,10.2397,0
2,12,603073.SH,20230206,20221231,1.2600,8.4560,0
3,13,001311.SZ,20230206,20221231,4.2000,31.6510,1
4,16,001225.SZ,20230404,20221231,4.5600,10.2397,1
...,...,...,...,...,...,...,...
1355,1800,301378.SZ,20230301,20221231,2.5400,13.4132,1
1356,1801,603065.SH,20230209,20221231,0.6500,4.7534,1
1357,1802,300062.SZ,20230405,20221231,0.0742,2.3533,1
1358,1803,300956.SZ,20230405,20221231,-0.2100,10.3967,1


In [44]:
# 根据目标period生成period_list

def get_TTM_period_list(target_period):
    if target_period[4:] != '1231':
        period_list = [target_period, str(int(target_period[:4]) - 1) + '1231',
                       str(int(target_period[:4]) - 1) + '0930']
    else:
        period_list = [target_period]
    return period_list

In [45]:
target_period = '20221231'
get_TTM_period_list(target_period)

['20221231']

In [46]:
target_period[4:]

'1231'

In [47]:
# 生成revenue-ps的TTM数据
indicator = 'revenue_ps'
indicators = [indicator]
period = '20220930'
period_list = get_TTM_period_list(period)
df = get_fin_indicators(period, indicators)
selected_sec_ls = df.ts_code.tolist()
if len(period_list) > 1:
    for period in period_list[1:]:
        new_df = get_fin_indicators(period, indicators)
        selected_sec_ls = list(set(selected_sec_ls) & set(new_df.ts_code.tolist()))
        df = pd.concat([df, new_df], ignore_index=True)
df = df.loc[df['ts_code'].isin(selected_sec_ls)]
df = df.reset_index()

In [51]:
# 针对公告修正进行调整
#使用 Pandas 库的 groupby() 函数将 Dataframe 中具有相同 col1 和 col2 值的行分组
groups = df.groupby(['ts_code', 'end_date'], group_keys=True)


def filter_group(group):
    if (group['update_flag'] == '0').all():
        return group
    return group[group['update_flag'] != '0']


df_filtered = groups.apply(filter_group)
df_filtered = df_filtered.set_index('index')
origin_sec_ls = list(set(df_filtered.ts_code.tolist()))

In [55]:
# 得到指定指标的TTM数据
indicator = 'revenue_ps'
result_df = pd.DataFrame(columns=['code', 'ann_date', 'period', indicator + '_TTM'])
i = 0
for sec in origin_sec_ls:
    print(i, end='\r')
    ann_date = df_filtered.loc[(df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period), indicator].iloc[0]
    indicator_data = \
        df_filtered.loc[(df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[0]), indicator].iloc[
            0] + \
        df_filtered.loc[(df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[1]), indicator].iloc[
            0] - \
        df_filtered.loc[(df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[2]), indicator].iloc[
            0]
    result_ls = [sec, ann_date, period, indicator_data]
    result_df.loc[i] = result_ls
    i += 1

In [56]:
result_df

,code,ann_date,end_date,revenue_ps
0,601333.SH,2.1102,20210930,2.8940
1,002629.SZ,0.2209,20210930,0.3765
2,002433.SZ,1.8861,20210930,1.9774
3,002901.SZ,3.2208,20210930,4.6621
4,002869.SZ,1.3186,20210930,2.3227
...,...,...,...,...
4686,000066.SZ,4.0394,20210930,4.9539
4687,301116.SZ,29.2571,20210930,41.0125
4688,300851.SZ,1.9182,20210930,4.2548
4689,002691.SZ,0.6144,20210930,0.9609


In [73]:
# 生成revenue-ps的TTM数据
threshold_date = '20230331'
indicator = 'revenue_ps'
target_period = '20221231'
indicators = [indicator]
period_list = get_TTM_period_list(target_period)
df = get_fin_indicators(target_period, indicators)
selected_sec_ls = df.ts_code.tolist()
if len(period_list) > 1:
    for period in period_list[1:]:
        new_df = get_fin_indicators(period, indicators)
        selected_sec_ls = list(set(selected_sec_ls) & set(new_df.ts_code.tolist()))
        df = pd.concat([df, new_df], ignore_index=True)
df = df.loc[df['ts_code'].isin(selected_sec_ls)]
df = df.reset_index()
# 针对公告修正进行调整
#使用 Pandas 库的 groupby() 函数将 Dataframe 中具有相同 col1 和 col2 值的行分组
groups = df.groupby(['ts_code', 'end_date'], group_keys=True)


def filter_group(group):
    if (group['update_flag'] == '0').all():
        return group
    return group[group['update_flag'] != '0']


df_filtered = groups.apply(filter_group)
df_filtered = df_filtered.set_index('index')
df_filtered = df_filtered.loc[df_filtered['ann_date'] <= threshold_date].copy()
origin_sec_ls = list(set(df_filtered.ts_code.tolist()))
# 得到指定指标的TTM数据
indicator = 'revenue_ps'
result_df = pd.DataFrame(columns=['code', 'ann_date', 'period', indicator + '_TTM'])
i = 0
for sec in origin_sec_ls:
    print(i, end='\r')
    ann_date = \
        df_filtered.loc[
            (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == target_period), 'ann_date'].iloc[0]
    if len(period_list) > 1:
        indicator_data = \
            df_filtered.loc[
                (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[0]), indicator].iloc[
                0] + \
            df_filtered.loc[
                (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[1]), indicator].iloc[
                0] - \
            df_filtered.loc[
                (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[2]), indicator].iloc[
                0]
    else:
        indicator_data = \
            df_filtered.loc[
                (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[0]), indicator].iloc[
                0]
    result_df.loc[i] = [sec, ann_date, target_period, indicator_data]
    i += 1


In [79]:
# 根据提取数据的日期生成此时已经发布截至的业绩报告期
def get_yj_period(date):
    year = date[:4]
    short_date = date[4:]
    if short_date <= '0430':
        yj_period = str(int(year) - 1) + '0930'
    elif short_date <= '0830':
        yj_period = year + '0331'
    elif short_date <= '1030':
        yj_period = year + '0630'
    else:
        yj_period = year + '0930'
    return yj_period

In [80]:
# 获取财务指标的TTM数据
def get_TTM_fin_data(target_period, threshold_date, indicator):
    indicators = [indicator]
    period_list = get_TTM_period_list(target_period)
    df = get_fin_indicators(target_period, indicators)
    selected_sec_ls = df.ts_code.tolist()
    if len(period_list) > 1:
        for period in period_list[1:]:
            new_df = get_fin_indicators(period, indicators)
            selected_sec_ls = list(set(selected_sec_ls) & set(new_df.ts_code.tolist()))
            df = pd.concat([df, new_df], ignore_index=True)
    df = df.loc[df['ts_code'].isin(selected_sec_ls)]
    df = df.reset_index()
    # 针对公告修正进行调整
    #使用 Pandas 库的 groupby() 函数将 Dataframe 中具有相同 col1 和 col2 值的行分组
    groups = df.groupby(['ts_code', 'end_date'], group_keys=True)

    def filter_group(group):
        if (group['update_flag'] == '0').all():
            return group
        return group[group['update_flag'] != '0']

    df_filtered = groups.apply(filter_group)
    df_filtered = df_filtered.set_index('index')
    df_filtered = df_filtered.loc[df_filtered['ann_date'] <= threshold_date].copy()
    origin_sec_ls = list(set(df_filtered.ts_code.tolist()))
    # 得到指定指标的TTM数据
    indicator = 'revenue_ps'
    result_df = pd.DataFrame(columns=['code', 'ann_date', 'period', indicator + '_TTM'])
    i = 0
    for sec in origin_sec_ls:
        print(i, end='\r')
        ann_date = \
            df_filtered.loc[
                (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == target_period), 'ann_date'].iloc[0]
        if len(period_list) > 1:
            indicator_data = \
                df_filtered.loc[
                    (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[0]), indicator].iloc[
                    0] + \
                df_filtered.loc[
                    (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[1]), indicator].iloc[
                    0] - \
                df_filtered.loc[
                    (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[2]), indicator].iloc[
                    0]
        else:
            indicator_data = \
                df_filtered.loc[
                    (df_filtered['ts_code'] == sec) & (df_filtered['end_date'] == period_list[0]), indicator].iloc[
                    0]
        result_df.loc[i] = [sec, ann_date, target_period, indicator_data]
        i += 1
    return result_df


threshold_date = '20230331'
indicator = 'revenue_ps'
target_period = get_yj_period(threshold_date)
get_TTM_fin_data(target_period, threshold_date, indicator)

,code,ann_date,period,revenue_ps_TTM
0,601333.SH,20221028,20220930,2.8940
1,002629.SZ,20221025,20220930,0.3765
2,002433.SZ,20221028,20220930,1.9774
3,002901.SZ,20221028,20220930,4.6621
4,002869.SZ,20221027,20220930,2.3227
...,...,...,...,...
4686,000066.SZ,20221029,20220930,4.9539
4687,301116.SZ,20221025,20220930,41.0125
4688,300851.SZ,20221027,20220930,4.2548
4689,002691.SZ,20221027,20220930,0.9609
